In [1]:
!pip install torch pandas
# Install PyTorch Geometric dependencies
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-geometric



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_list = torch.load("/workspace/test_graph.pt")

print("Jumlah grafik:", len(data_list))
print("Tipe objek pertama:", type(data_list[0]))
print("Isi atribut:", data_list[0])

In [ ]:
import torch
import pandas as pd

# Load dataset dan file referensi
graph_data = torch.load("/content/drive/MyDrive/Hasil_Ekstraksi/cnn_layout/Fitur_cnn_layout.pt", weights_only=False)


# Load CSV
train_df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/80_20/train_80_20.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/80_20/test_80_20.csv")

# Ambil daftar nama file dari CSV
train_filenames = set(train_df['image'].values)
test_filenames = set(test_df['image'].values)

# Pisahkan data
train_graphs = []
test_graphs = []
other_graphs = []

for graph in graph_data:
    filename = graph.get('file_name')  # atau sesuaikan dengan key yang digunakan
    if filename in train_filenames:
        train_graphs.append(graph)
    elif filename in test_filenames:
        test_graphs.append(graph)
    else:
        other_graphs.append(graph)  # Jika tidak ada di dua csv

# Simpan hasil
torch.save(train_graphs, "/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/train_cnn_layout.pt")
torch.save(test_graphs, "/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/test_cnn_layout.pt")



In [ ]:
import torch
import pandas as pd


# Ambil nama file dari masing-masing sumber
graph_filenames = set(graph.get('file_name') for graph in train_graphs)
csv_filenames = set(train_df['image'].values)

# Cek kesesuaian
missing_in_graphs = csv_filenames - graph_filenames
extra_in_graphs = graph_filenames - csv_filenames

print("Jumlah file di CSV:", len(csv_filenames))
print("Jumlah file di train_graphs:", len(graph_filenames))
print("File yang ada di CSV tapi tidak di train_graphs:", missing_in_graphs)
print("File yang ada di train_graphs tapi tidak di CSV:", extra_in_graphs)


Jumlah file di CSV: 19200
Jumlah file di train_graphs: 19200
File yang ada di CSV tapi tidak di train_graphs: set()
File yang ada di train_graphs tapi tidak di CSV: set()


In [ ]:
import torch

# Path file asal
train_path = "/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/train_cnn_layout.pt"
val_path = "/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/val_cnn_layout.pt"

# Load data
data = torch.load(train_path,weights_only=False)

# Tentukan panjang data
total_len = len(data)
val_len = int(0.2 * total_len)

# Acak index
indices = torch.randperm(total_len)
val_indices = indices[:val_len]
train_indices = indices[val_len:]

# Pisahkan data berdasarkan tipe
if isinstance(data, list):
    val_data = [data[i] for i in val_indices]
    train_data = [data[i] for i in train_indices]
elif isinstance(data, torch.Tensor):
    val_data = data[val_indices]
    train_data = data[train_indices]
else:
    raise TypeError("Isi file .pt tidak dikenali (bukan list atau tensor)")

# Simpan ulang
torch.save(train_data, train_path)
torch.save(val_data, val_path)

print("✅ Data berhasil dipisahkan:")
print(f"- Train (80%) disimpan ke: {train_path}")
print(f"- Validasi (20%) disimpan ke: {val_path}")


✅ Data berhasil dipisahkan:
- Train (80%) disimpan ke: /content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/train_cnn_layout.pt
- Validasi (20%) disimpan ke: /content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/val_cnn_layout.pt


In [ ]:
print(f"- Train (80%): {len(train_data)} item -> disimpan ke: {train_path}")
print(f"- Validasi (20%): {len(val_data)} item -> disimpan ke: {val_path}")

- Train (80%): 15360 item -> disimpan ke: /content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/train_cnn_layout.pt
- Validasi (20%): 3840 item -> disimpan ke: /content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/val_cnn_layout.pt


In [ ]:
import torch

# Path file input & output
input_path = "/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/val_cnn_layout.pt"
output_path = input_path  # bisa diubah kalau mau simpan ke file berbeda

# Load data
data_list = torch.load(input_path, weights_only=False)

# Perbaiki format edge_index jika perlu
for data in data_list:
    if hasattr(data, 'edge_index') and data.edge_index.shape[0] != 2:
        data.edge_index = data.edge_index.T  # transpose ke [2, num_edges]

# Simpan kembali data
torch.save(data_list, output_path)

print(f"✅ Semua edge_index diperiksa dan disimpan ulang ke '{output_path}'")


✅ Semua edge_index diperiksa dan disimpan ulang ke '/content/drive/MyDrive/Hasil_Ekstraksi/80_20/cnn_layout/val_cnn_layout.pt'


In [ ]:
# Verifikasi semua edge_index sudah benar
for i, data in enumerate(data_list):
    if not hasattr(data, 'edge_index') or data.edge_index.shape[0] != 2:
        print(f"❌ Format edge_index salah di index {i}")
    else:
        assert data.edge_index.shape[0] == 2, "edge_index masih salah bentuk"
print("✅ Semua edge_index sudah dalam bentuk [2, num_edges]")


✅ Semua edge_index sudah dalam bentuk [2, num_edges]


In [ ]:
# Path file input & output
input_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color/train_color_tata_letak_only.pt"
output_path = input_path  # bisa diubah kalau mau simpan ke file berbeda

# Load data
data_list = torch.load(input_path, weights_only=False)

print("Jumlah grafik:", len(data_list))
print("Tipe objek pertama:", type(data_list[0]))
print("Isi atribut:", data_list[0])

Jumlah grafik: 17280
Tipe objek pertama: <class 'torch_geometric.data.data.Data'>
Isi atribut: Data(x=[9, 12], edge_index=[2, 12], y=0, file_name='16223.jpg')


In [ ]:
# --- 1. BACA CSV ---
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv")


In [ ]:
import torch

# Path file
first_path = "/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv"
second_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color/processed_graphs_color_tata_letak_only.pt"

# ==== 1. Baca hanya header CSV ====
with open(first_path, 'r', encoding='utf-8') as f:
    header_line = f.readline().strip()
    headers = header_line.split(',')

print("📄 Header dari CSV:")
print(headers)

# ==== 2. Baca file .pt ====
data_list = torch.load(second_path,weights_only=False)

print("\n📦 Jumlah graph dalam .pt:", len(data_list))
print("📦 Atribut dari graph pertama:")
print(data_list[0])


📄 Header dari CSV:
['Filename', 'Category', 'W', 'H', 'Original_Width', 'Original_Height', 'Node_Features', 'Edge_Index', 'Color_Features']


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "



📦 Jumlah graph dalam .pt: 24010
📦 Atribut dari graph pertama:
Data(x=[9, 12], edge_index=[12, 2], y=0, file_name='13089.jpg')


In [ ]:
import pandas as pd
import torch
import json

# Path file
first_path = "/content/drive/MyDrive/Hasil_Ekstraksi/graph_dataset_fresh.csv"
second_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color/processed_graphs_color_tata_letak_only.pt"

# ==== 1. Baca CSV lalu ubah ke dict ====
df = pd.read_csv(first_path)
csv_as_dict = df.to_dict(orient='records')  # List of dicts (seperti JSON array of objects)

# Cetak keys dari satu record (header)
print("📄 Header dari CSV (pakai json-like dict):")
print(list(csv_as_dict[0].keys()))

# ==== 2. Baca file .pt ====
data_list = torch.load(second_path)

print("\n📦 Jumlah graph dalam .pt:", len(data_list))
print("📦 Atribut (field) dari graph pertama:")
print(data_list[0])


In [ ]:
import pandas as pd
import ast

# Path ke file CSV kamu
csv_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color_only/csv_color_only.csv"

# Daftar gambar target
target_ids = ['1', '41', '49', '58', '6', '80', '111', '136', '747', '201']
target_filenames = [f"{i}.jpg" for i in target_ids]

# Baca CSV
df = pd.read_csv(csv_path)

# Filter berdasarkan nama file
filtered_df = df[df['Filename'].isin(target_filenames)].copy()

# Fungsi untuk mencetak Node 0–2 dengan format sesuai permintaan
def print_first_3_nodes(row):
    try:
        node_features = ast.literal_eval(row['Node_Features'])
        print(f"\n📁 {row['Filename']}")
        for i in range(min(3, len(node_features))):
            print(f"Node {i} : {node_features[i]}")
    except Exception as e:
        print(f"❌ Error di {row['Filename']}: {e}")

# Jalankan fungsi untuk setiap baris
filtered_df.apply(print_first_3_nodes, axis=1)



📁 1.jpg
Node 0 : [186.61512756347656, 14.533888816833496, -3.4412951469421387, 167.55284118652344, 16.898588180541992, -0.5582130551338196, 159.79339599609375, 22.448741912841797, -0.6580144762992859]
Node 1 : [193.02734375, 7.811577320098877, 0.11124493926763535, 166.0377655029297, 37.342525482177734, -2.6271417140960693, 162.84060668945312, 35.506622314453125, -2.0490477085113525]
Node 2 : [194.60665893554688, 6.501161575317383, 0.779297947883606, 175.8079376220703, 16.364727020263672, 0.6768613457679749, 170.77615356445312, 17.75863265991211, 0.6945494413375854]

📁 111.jpg
Node 0 : [190.6194610595703, 42.467041015625, -0.6714568734169006, 155.77288818359375, 46.5402717590332, -0.3380005955696106, 80.43966674804688, 46.69710922241211, 0.5541598796844482]
Node 1 : [196.56793212890625, 26.705110549926758, -0.5855464935302734, 159.73968505859375, 30.452590942382812, -0.15674076974391937, 81.47797393798828, 31.234582901000977, 0.4375627636909485]
Node 2 : [159.41796875, 37.7646484375, -

,0
10910,None
12134,None
14400,None
15122,None
17441,None
18328,None
19327,None
19548,None
21181,None
21770,None


In [ ]:
import torch

# ==== Ubah sesuai lokasi file kamu ====
pt_path = "/content/drive/MyDrive/Hasil_Ekstraksi/color_only/processed_graphs_color__only_normal.pt"

# Target file_name yang ingin dicek
target_ids = ['1', '41', '49', '58', '6', '80', '111', '136', '747', '201']
target_filenames = [f"{i}.jpg" for i in target_ids]

# Load file .pt (berisi list of Data)
print("🔄 Loading .pt file...")
data_list = torch.load(pt_path, map_location='cpu',weights_only=False)
print(f"✅ Loaded {len(data_list)} graphs")

# Buat dict: file_name -> data
file_to_graph = {
    data.file_name: data
    for data in data_list
    if hasattr(data, 'file_name')
}

# Loop tiap target dan cetak fitur Node 0–2
for fname in target_filenames:
    print(f"\n📁 {fname}")
    graph = file_to_graph.get(fname)

    if graph is None:
        print("❌ Graph tidak ditemukan.")
        continue

    try:
        x = graph.x
        for i in range(min(3, x.shape[0])):
            rounded = [round(val.item(), 4) for val in x[i]]
            print(f"Node {i}: {rounded}")
    except Exception as e:
        print(f"❌ Error membaca node: {e}")


🔄 Loading .pt file...
✅ Loaded 24010 graphs

📁 1.jpg
Node 0: [0.906, -1.6586, -2.8006, 1.0022, -1.6291, -0.5587, 1.2596, -1.1935, -0.9246]
Node 1: [1.0393, -2.0452, 0.3345, 0.9699, -0.4177, -2.4415, 1.3212, -0.4918, -2.077]
Node 2: [1.0721, -2.1206, 0.9241, 1.1783, -1.6607, 0.5653, 1.4817, -1.4456, 0.1959]

📁 41.jpg
Node 0: [0.8081, -0.7339, 0.6316, 0.5797, 0.6044, 0.3697, 0.8848, 0.7416, -0.0401]
Node 1: [0.5074, 1.0027, -0.1758, 0.2655, 2.4126, -0.2618, 0.46, 2.4841, -0.4527]
Node 2: [0.4498, 0.1516, -0.7923, -0.023, 0.9178, 0.1395, 0.2298, 1.0171, -0.1521]

📁 49.jpg
Node 0: [-0.3255, 1.2271, -0.2115, -0.1184, 1.317, -0.4696, 0.0922, 1.0057, -0.6094]
Node 1: [0.2254, 1.6237, -0.402, 0.2563, 1.7141, -0.6689, 0.2645, 1.0399, -0.7436]
Node 2: [-0.6684, 1.293, -0.0738, -0.5911, 1.4227, -0.1989, -0.3265, 0.8641, -0.4299]

📁 58.jpg
Node 0: [-1.9453, 0.0094, 0.2752, -1.7257, -0.5399, 0.2102, -1.4626, -1.0351, 0.876]
Node 1: [-2.0244, 0.0041, 0.2319, -1.8025, -0.617, -0.0567, -1.5248, -1.272

In [5]:
import os
import pandas as pd
from IPython.display import display

# Path file
base_path = '/content/drive/MyDrive/Hasil_Ekstraksi/'

# Baca file CSV
df_train_80_20 = pd.read_csv(base_path + '80_20/train_80_20.csv')
df_test_80_20 = pd.read_csv(base_path + '80_20/test_80_20.csv')
df_train = pd.read_csv(base_path + 'train.csv')
df_test = pd.read_csv(base_path + 'test.csv')

# Fungsi normalisasi nama file
def normalize_filename(series):
    return set(
        series.astype(str)
              .str.strip()
              .str.lower()
              .apply(lambda x: os.path.basename(x))  # ambil hanya nama file tanpa path
    )

# Normalisasi dan konversi ke set
set_train_80_20 = normalize_filename(df_train_80_20['image'])
set_test_80_20  = normalize_filename(df_test_80_20['image'])
set_train       = normalize_filename(df_train['image'])
set_test        = normalize_filename(df_test['image'])

# Cari irisan
common_images = set_train_80_20 & set_test_80_20 & set_train & set_test

# Simpan ke CSV dan tampilkan
common_df = pd.DataFrame({'common_image': sorted(list(common_images))})
common_df.to_csv(base_path + 'common_images.csv', index=False)

print(f"Total nama file image yang sama di semua file: {len(common_df)}")
display(common_df.head())


Total nama file image yang sama di semua file: 0


,common_image
